## Import

In [1]:
import random
import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split

/home/jihui/miniconda3/envs/dacon/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
pd.read_csv('./dataset/ETTh1.csv')

,date,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT
0,2016-07-01 00:00:00,5.827,2.009,1.599,0.462,4.203,1.340,30.531000
1,2016-07-01 01:00:00,5.693,2.076,1.492,0.426,4.142,1.371,27.787001
2,2016-07-01 02:00:00,5.157,1.741,1.279,0.355,3.777,1.218,27.787001
3,2016-07-01 03:00:00,5.090,1.942,1.279,0.391,3.807,1.279,25.044001
4,2016-07-01 04:00:00,5.358,1.942,1.492,0.462,3.868,1.279,21.948000
...,...,...,...,...,...,...,...,...
17415,2018-06-26 15:00:00,-1.674,3.550,-5.615,2.132,3.472,1.523,10.904000
17416,2018-06-26 16:00:00,-5.492,4.287,-9.132,2.274,3.533,1.675,11.044000
17417,2018-06-26 17:00:00,2.813,3.818,-0.817,2.097,3.716,1.523,10.271000
17418,2018-06-26 18:00:00,9.243,3.818,5.472,2.097,3.655,1.432,9.778000


## Hyperparameter Setting

In [4]:
CFG = {
    'TRAIN_WINDOW_SIZE':90, # 90일치로 학습
    'PREDICT_SIZE':21, # 21일치 예측
    'EPOCHS':10,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':4096,
    'SEED':41,
    'LAMBDA':0.8,
    'LR_LAMBDA':0.70
}

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

### 데이터 불러오기

- Encode Info

In [6]:
train_data = pd.read_csv('./train.csv').drop(columns=['ID', '제품'])

In [7]:
output_kuka = train_data.iloc[:,-CFG['PREDICT_SIZE']:].to_numpy()

- Product Features

In [8]:
product_features = pd.read_csv('./features.csv').drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'ID', '제품', '대분류', '중분류', '소분류', '브랜드'])

- Time Series Features

In [9]:
time_series_features = pd.read_csv('./total_dates_scaling.csv').drop(columns=['Unnamed: 0', 'Date'])

- Product & Time Series Features

In [10]:
# (15890, 459)
# price = pd.read_csv('./price.csv').iloc[:,7:]
# sale_event = pd.read_csv('./sale_event.csv').drop(columns=['Unnamed: 0'])

### 데이터 전처리

- Encode Info

In [11]:
# 숫자형 변수들의 min-max scaling을 수행하는 코드입니다.
numeric_cols = train_data.columns[4:]
# 각 column의 min 및 max 계산
min_values = train_data[numeric_cols].min(axis=1)
max_values = train_data[numeric_cols].max(axis=1)
# 각 행의 범위(max-min)를 계산하고, 범위가 0인 경우 1로 대체
ranges = max_values - min_values
ranges[ranges == 0] = 1
# min-max scaling 수행
train_data[numeric_cols] = (train_data[numeric_cols].subtract(min_values, axis=0)).div(ranges, axis=0)
# max와 min 값을 dictionary 형태로 저장
scale_min_dict = min_values.to_dict()
scale_max_dict = max_values.to_dict()

In [12]:
# Label Encoding
label_encoder = LabelEncoder()
categorical_columns = ['대분류', '중분류', '소분류', '브랜드']
categorical_nums = [5, 11, 53, 3170]

for i, col in enumerate(categorical_columns):
    label_encoder.fit(train_data[col])
    train_data[col] = label_encoder.transform(train_data[col])
    train_data[col] = train_data[col].div(categorical_nums[i]-1) # minmax scaling


- Product Features

In [13]:
# (15890, 1)로 만들기
scaler1 = MinMaxScaler()
scaler1.fit(product_features[['판매량평균']])
sales_mean_scaled = scaler1.transform(product_features[['판매량평균']])

scaler2 = MinMaxScaler()
scaler2.fit(product_features[['제품수']])
prod_num_scaled = scaler2.transform(product_features[['제품수']])

nprice_mid = product_features[['NormalizedPrice_중분류']].to_numpy()

In [14]:
product = np.column_stack((sales_mean_scaled, prod_num_scaled, nprice_mid))

- Time Series Features

In [15]:
# (459, 1)
# weekend_salary = (time_series_features[['Weekend']].to_numpy() + 0.5*time_series_features[['Salary']].to_numpy() + 0.5*np.where(time_series_features[['DayofWeek']]==4/6, 1, 0)).squeeze()
sale_info = time_series_features['SaleInfo'].to_numpy()
holiday = time_series_features['Holiday'].to_numpy()
salary = time_series_features['Salary'].to_numpy()
month = time_series_features['Month'].to_numpy()
dayofweek = time_series_features['DayofWeek'].to_numpy()

In [16]:
time_series = np.stack((sale_info, salary, month, dayofweek)) 

In [17]:
time_series.shape

(4, 459)

In [18]:
def make_train_data(data, train_size=CFG['TRAIN_WINDOW_SIZE'], predict_size=CFG['PREDICT_SIZE']):
    '''
    학습 기간 블럭, 예측 기간 블럭의 세트로 데이터를 생성
    data : 일별 판매량
    train_size : 학습에 활용할 기간
    predict_size : 추론할 기간
    '''
    num_rows = len(data)
    window_size = train_size + predict_size
    
    input_data = np.empty((num_rows * (len(data.columns) - window_size + 1), train_size, 1 + product.shape[1] + time_series.shape[0] + 1))
    target_data = np.empty((num_rows * (len(data.columns) - window_size + 1), predict_size, 1 + product.shape[1] + time_series.shape[0] + 1))
    # target_data = np.empty((num_rows * (len(data.columns) - window_size + 1), predict_size))
    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, 0]) # 대분류 소분류
        product_data = product[i,:]
        # time_series
        sales_data = np.array(data.iloc[i, 4:])
        # price_data = np.array(price.iloc[i, :])
        # sale_event_data = np.array(sale_event.iloc[i, :])
        
        for j in range(len(sales_data) - window_size + 1):
            time_series_window = time_series[:, j : j + window_size]
            # price_window = price_data[j : j + window_size]
            # sale_event_window = sale_event_data[j : j + window_size]
            window = sales_data[j : j + window_size]
            
            temp_data = np.concatenate((np.tile(encode_info, (train_size, 1)), np.tile(product_data, (train_size, 1)), time_series_window[:,:train_size].T,window[:train_size].reshape((-1,1))), axis=1)
            temp_target = np.concatenate((np.tile(encode_info, (predict_size, 1)), np.tile(product_data, (predict_size, 1)), time_series_window[:,train_size:].T, window[train_size:].reshape((-1,1))), axis=1)
            input_data[i * (len(data.columns) - window_size + 1) + j] = temp_data
            target_data[i * (len(data.columns) - window_size + 1) + j] = temp_target
    
    return input_data, target_data

In [20]:
def make_predict_data(data, train_size=CFG['TRAIN_WINDOW_SIZE']):
    '''
    평가 데이터(Test Dataset)를 추론하기 위한 Input 데이터를 생성
    data : 일별 판매량
    train_size : 추론을 위해 필요한 일별 판매량 기간 (= 학습에 활용할 기간)
    '''
    num_rows = len(data)
    
    input_data = np.empty((num_rows, train_size, 1 + product.shape[1] + time_series.shape[0] + 1))
    
    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, 0])
        product_data = product[i,:]
        sales_data = np.array(data.iloc[i, -train_size:])
        # price_data = np.array(price.iloc[i, -train_size:])
        # sale_event_data = np.array(sale_event.iloc[i, -train_size:])
        
        time_series_window = time_series[:, -train_size:]
        # price_window = price_data[-train_size:]
        # sale_event_window = sale_event_data[-train_size:]
        window = sales_data[-train_size : ]

        temp_data = np.concatenate((np.tile(encode_info, (train_size, 1)), np.tile(product_data, (train_size, 1)), time_series_window[:,:train_size].T, window[:train_size].reshape((-1,1))),axis=1)
        input_data[i] = temp_data
    
    return input_data

In [21]:
# train_input, train_target = make_train_data(train_data) # train val total
train_input, train_target = make_train_data(train_data.iloc[:,:-CFG['PREDICT_SIZE']]) # train val
# test_input_kuka = make_predict_data(train_data.iloc[:,:-CFG['PREDICT_SIZE']]) # test(kuka pre-submission)
# test_input = make_predict_data(train_data) # test(submission)

100%|██████████| 15890/15890 [02:15<00:00, 117.56it/s]


### Custom Dataset

In [22]:
class CustomDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        
    def __getitem__(self, index):
        if self.Y is not None:
            return torch.Tensor(self.X[index]), torch.Tensor(self.Y[index])
        return torch.Tensor(self.X[index])
    
    def __len__(self):
        return len(self.X)

In [23]:
dataset = CustomDataset(train_input, train_target)

data_len = len(dataset)
train_size = int(data_len*0.8)
val_size = int(data_len*0.2)

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
# train_loader_iter = enumerate(train_loader)
# i, batch_train = train_loader_iter.__next__()

# X, Y = batch_train
# print(X.shape)
# print(Y.shape)

## Prepare the forecaster 🏋️‍♂️

### 모델 선언

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, input_size=13, hidden_size=512, output_size=CFG['PREDICT_SIZE']):
        super(BaseModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size//2),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(hidden_size//2, output_size)
        )
            
        self.actv = nn.ReLU()
    
    def forward(self, x):
        # x shape: (B, TRAIN_WINDOW_SIZE, 13)
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size, x.device)
        
        # LSTM layer
        lstm_out, hidden = self.lstm(x, hidden)
        
        # Only use the last output sequence
        last_output = lstm_out[:, -1, :]
        
        # Fully connected layer
        output = self.actv(self.fc(last_output))
        
        return output.squeeze(1)
    
    def init_hidden(self, batch_size, device):
        # Initialize hidden state and cell state
        return (torch.zeros(1, batch_size, self.hidden_size, device=device),
                torch.zeros(1, batch_size, self.hidden_size, device=device))

### 모델 학습

In [ ]:
class PSFALoss(nn.Module):
    def __init__(self):
        super(PSFALoss, self).__init__()

    def forward(self, inputs, targets):
        share_denominator = torch.sum(targets, axis=0)
        share_denominator[share_denominator==float(0)] = 1 # 나눠지게 만들자
        share = targets/share_denominator
        error_demoninator = torch.max(inputs, targets)
        error_demoninator[error_demoninator==float(0)] = 1 # 나눠지게 만들자
        error = torch.abs(inputs-targets)/error_demoninator
        metric = error * share
        loss = torch.mean(torch.sum(metric, axis=1))
        return loss

In [ ]:
indexs_bigcat={}
for bigcat in train_data['대분류'].unique():
    indexs_bigcat[bigcat] = list(train_data.loc[train_data['대분류']==bigcat].index)

indexs_bigcat.keys()

In [ ]:
def PSFA(pred, target): 
    PSFA = 1
    for cat in range(5):
        ids = indexs_bigcat[cat]
        for day in range(21):
            total_sell = np.sum(target[ids, day]) # day별 총 판매량
            pred_values = pred[ids, day] # day별 예측 판매량
            target_values = target[ids, day] # day별 실제 판매량
            
            # 실제 판매와 예측 판매가 같은 경우 오차가 없는 것으로 간주 
            denominator = np.maximum(target_values, pred_values)
            diffs = np.where(denominator!=0, np.abs(target_values - pred_values) / denominator, 0)
            
            if total_sell != 0:
                sell_weights = target_values / total_sell  # Item별 day 총 판매량 내 비중
            else:
                sell_weights = np.ones_like(target_values) / len(ids)  # 1 / len(ids)로 대체
                
            if not np.isnan(diffs).any():  # diffs에 NaN이 없는 경우에만 PSFA 값 업데이트
                PSFA -= np.sum(diffs * sell_weights) / (21 * 5)
            
            
    return PSFA

In [ ]:
def train(model, optimizer, scheduler, train_loader, val_loader, device):
    model.to(device)
    criterion = nn.MSELoss().to(device)
    psfa = PSFALoss().to(device)
    best_loss = 9999999
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        train_mae = []
        for X, Y in tqdm(iter(train_loader)):
            X = X.to(device)
            Y = Y.to(device)
            
            optimizer.zero_grad()
            
            output = model(X)
            
            loss = (1-CFG['LAMBDA'])*criterion(output, Y) + CFG['LAMBDA']*psfa(output, Y)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
        scheduler.step() # added
        
        val_loss = validation(model, val_loader, criterion, psfa, device)
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{val_loss:.5f}]')
        
        if best_loss > val_loss:
            best_loss = val_loss
            best_model = model
            print('Model Saved')
    return best_model

In [ ]:
def validation(model, val_loader, criterion, psfa, device):
    model.eval()
    val_loss = []
    
    with torch.no_grad():
        for X, Y in tqdm(iter(val_loader)):
            X = X.to(device)
            Y = Y.to(device)
            
            output = model(X)
            
            loss = (1-CFG['LAMBDA'])*criterion(output, Y) + CFG['LAMBDA']*psfa(output, Y)
            
            val_loss.append(loss.item())
    return np.mean(val_loss)

## Run !!

In [ ]:
model = BaseModel()
print(torch.cuda.device_count())
model = nn.DataParallel(model) # parallel mode
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = optim.lr_scheduler.LambdaLR(optimizer=optimizer,
                                        lr_lambda=lambda epoch: CFG["LR_LAMBDA"] ** epoch)
infer_model = train(model, optimizer, scheduler, train_loader, val_loader, device)

- 모델 저장

In [ ]:
torch.save(infer_model.state_dict, './baseline_submit_0818_ver1.pth')

## 모델 추론

- 저장된 파일 불러오기

In [ ]:
#### 추가
infer_model = BaseModel().to(device)
print(torch.cuda.device_count())
infer_model = nn.DataParallel(infer_model) # parallel mode

infer_model.load_state_dict(torch.load('./baseline_submit_0818_ver1.pth'))

In [ ]:
# 제출용
test_dataset = CustomDataset(test_input, None)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
# 성능확인용(추가)
test_dataset_kuka = CustomDataset(test_input_kuka, None)
test_loader_kuka = DataLoader(test_dataset_kuka, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
# test_loader_iter = enumerate(test_loader)
# i, batch_test = test_loader_iter.__next__()
# print(i, batch_test)
# X = batch_test
# print(X.shape)

In [ ]:
def inference(model, test_loader, device):
    predictions = []
    
    with torch.no_grad():
        for X in tqdm(iter(test_loader)):
            X = X.to(device)
            
            output = model(X)
            
            # 모델 출력인 output을 CPU로 이동하고 numpy 배열로 변환
            output = output.cpu().numpy()
            
            predictions.extend(output)
    
    return np.array(predictions)

### 성능확인용

In [ ]:
pred_kuka = inference(infer_model, test_loader_kuka, device)

In [ ]:
indexs_bigcat={}
for bigcat in train_data['대분류'].unique():
    indexs_bigcat[bigcat] = list(train_data.loc[train_data['대분류']==bigcat].index)

indexs_bigcat.keys()

In [ ]:
# 대분류 scaling 때문에 추가
d = {0.25:1, 0.5:2, 0.0:0, 1.0:4, 0.75:3}
indexs_bigcat = dict((d[key], value) for (key, value) in indexs_bigcat.items())

In [ ]:
# 추론 결과를 inverse scaling
for idx in range(len(pred_kuka)):
    pred_kuka[idx, :] = pred_kuka[idx, :] * (scale_max_dict[idx] - scale_min_dict[idx]) + scale_min_dict[idx]
    
# 결과 후처리
pred_kuka = np.round(pred_kuka, 0).astype(int)

In [ ]:
print(PSFA(pred_kuka, output_kuka))

### 제출용

In [ ]:
pred = inference(infer_model, test_loader, device)

In [ ]:
# 추론 결과를 inverse scaling
for idx in range(len(pred)):
    pred[idx, :] = pred[idx, :] * (scale_max_dict[idx] - scale_min_dict[idx]) + scale_min_dict[idx]
    
# 결과 후처리
pred = np.round(pred, 0).astype(int)

In [ ]:
pred.shape

## Submission

In [ ]:
submit = pd.read_csv('./sample_submission.csv')
submit.head()

In [ ]:
submit.iloc[:,1:] = pred
submit.head()

In [ ]:
submit.to_csv('./baseline_submit_0818_ver1.csv', index=False)